In [1]:
!pip3 install pandas numpy

import pandas as pd
import numpy as np

df_airports = pd.read_csv('./airports.csv')
df_airlines = pd.read_csv('./airlines.csv')
df_flights = pd.read_csv('./flights.csv')

/var/folders/84/flln9myn2m19wbd93vxbm8680000gn/T/ipykernel_2783/1867893514.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv('./flights.csv')


In [ ]:
#exercise 1
df_flights = df_flights.rename(columns={"AIRLINE":"IATA_CODE"})

df_flights = pd.merge(df_flights,df_airlines,on="IATA_CODE",how="inner")

df_select_flights =  df_flights[(df_flights["YEAR"] == 2015) & (df_flights["MONTH"] == 6)]

df_flights_group = df_select_flights.groupby('AIRLINE').agg({"FLIGHT_NUMBER":"count"}).sort_values(by='FLIGHT_NUMBER',ascending=[False])

df_flights_group

In [5]:
#exercise 2

df_airports = pd.read_csv('./airports.csv')
df_airlines = pd.read_csv('./airlines.csv')
df_flights = pd.read_csv('./flights.csv')


def is_delay(x):
    if(x > 0):
        return 1
    else:
        return 0

df_flights_delay_min = df_flights.copy()

df_flights["DEPARTURE_DELAY"] = df_flights["DEPARTURE_DELAY"].apply(is_delay)

df_flights_grouped = df_flights.groupby(["AIRLINE","ORIGIN_AIRPORT"]).agg({"FLIGHT_NUMBER":"count","CANCELLED":"sum","SCHEDULED_DEPARTURE":"count","DEPARTURE_DELAY":"sum"})

df_flights_grouped["OPERATE_RATE"] = (df_flights_grouped["FLIGHT_NUMBER"] - df_flights_grouped["CANCELLED"]) /  df_flights_grouped["FLIGHT_NUMBER"] * 100

df_flights_grouped["DELAY_RATE"] = df_flights_grouped["DEPARTURE_DELAY"] / (df_flights_grouped["SCHEDULED_DEPARTURE"] - df_flights_grouped["CANCELLED"]) * 100

df_flights_grouped = df_flights_grouped.drop(columns=["CANCELLED","SCHEDULED_DEPARTURE","DEPARTURE_DELAY"]).rename(columns={"FLIGHT_NUMBER":"TOTAL_NUMBER_FLIGHT"})

df_flights_delay_min_grouped = df_flights_delay_min[df_flights_delay_min["DEPARTURE_DELAY"] > 0].groupby(["AIRLINE","ORIGIN_AIRPORT"]).agg({"DEPARTURE_DELAY":"mean"}).rename(columns={"DEPARTURE_DELAY":"AVERAGE_DELAY_MINUTES"})

df_flight_table = pd.concat([df_flights_grouped,df_flights_delay_min_grouped],axis=1)

df_flight_table

/var/folders/84/flln9myn2m19wbd93vxbm8680000gn/T/ipykernel_2783/2796968586.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv('./flights.csv')


TOTAL_NUMBER_FLIGHT  OPERATE_RATE  DELAY_RATE  \
AIRLINE ORIGIN_AIRPORT                                                  
AA      10140                           174     97.701149   25.294118   
        10257                            80    100.000000   21.250000   
        10299                            32    100.000000   25.000000   
        10397                           917     99.236641   26.263736   
        10423                           786     98.091603   23.086900   
...                                     ...           ...         ...   
WN      SNA                           17197     99.127755   47.545023   
        STL                           27753     98.890210   44.765895   
        TPA                           25888     99.107695   40.651674   
        TUL                            4098     99.243533   37.964101   
        TUS                            4201     99.523923   35.493901   

                        AVERAGE_DELAY_MINUTES  
AIRLINE ORIGIN_AIRPORT                         
AA      10140                       43.209302  
        10257                       27.764706  
        10299                        8.125000  
        10397                       22.991632  
        10423                       42.050562  
...                                       ...  
WN      SNA                         20.307465  
        STL                         28.778121  
        TPA                         30.291083  
        TUL                         30.478627  
        TUS                         27.552561  

[3437 rows x 4 columns]